### Q1

In [10]:
import networkx as nx
import numpy as np
import matlab.engine

# Helper function to load and analyze a graph
def analyze_graph(file_path, network_name):
    # Step 1: Load the undirected graph using networkx
    G_nx = nx.read_edgelist(file_path, create_using=nx.Graph(), nodetype=int)

    # Step 2: Check the basic properties of the graph
    num_nodes = G_nx.number_of_nodes()
    num_edges = G_nx.number_of_edges()
    
    # Degree statistics
    degrees = np.array([deg for (node, deg) in G_nx.degree()])
    max_degree = np.max(degrees)
    min_degree = np.min(degrees)
    avg_degree = np.mean(degrees)

    # Degree distribution type (simplified: scale-free or unknown)
    if np.max(degrees) > 10 * np.mean(degrees):  # rough heuristic for scale-free
        degree_dist_type = 'Scale-free'
    else:
        degree_dist_type = 'Unknown'

    # Step 3: Check for connectivity (undirected graph)
    is_connected = nx.is_connected(G_nx)
    if not is_connected:
        diameter = 'N/A - Graph not connected'
    else:
        diameter = 'Good'  # Diameter only makes sense for connected graphs
        

    # Print out the required information
    print(f"--- {network_name} ---")
    print(f"n (Number of nodes): {num_nodes}")
    print(f"m (Number of edges): {num_edges}")
    print(f"min(d) (Minimum degree): {min_degree}")
    print(f"max(d) (Maximum degree): {max_degree}")
    print(f"avg(d) (Average degree): {avg_degree:.2f}")
    print(f"Degree distribution type: {degree_dist_type}")
    print(f"Diameter: {diameter}")
    print()

    # Step 4: Compute the Local Clustering Coefficient (LCC) for each node
    lcc_dict = nx.clustering(G_nx)
    lcc_values = np.array(list(lcc_dict.values()))  # Convert to numpy array

    return degrees, lcc_values

# Helper function to plot distributions using MATLAB
def plot_with_matlab(eng, degree, lcc_values, network_name):
    nbins = 50
    
    # Plot Degree Distribution
    eng.workspace['d'] = degree
    figID = 1
    eng.eval(f"plot_distribution(d, '{network_name} Degree Distribution', {nbins}, {figID})", nargout=0)
    
    # Plot LCC Distribution
    eng.workspace['lcc'] = lcc_values
    figID = 2
    eng.eval(f"plot_distribution(lcc, '{network_name} LCC Distribution', {nbins}, {figID})", nargout=0)


In [7]:

eng = matlab.engine.start_matlab()
eng.eval("addpath(genpath('Mcodes'))", nargout=0)

file_path_amazon = 'data/com-amazon.ungraph/com-amazon.ungraph.txt'
degree_amazon, lcc_amazon = analyze_graph(file_path_amazon, "Amazon")
plot_with_matlab(eng, degree_amazon, lcc_amazon, "Amazon")

--- Amazon ---
n (Number of nodes): 334863
m (Number of edges): 925872
min(d) (Minimum degree): 1
max(d) (Maximum degree): 549
avg(d) (Average degree): 5.53
Degree distribution type: Scale-free
Diameter: Good



In [3]:

file_path_facebook = 'data/facebook/facebook_combined.txt'
degree_facebook, lcc_facebook = analyze_graph(file_path_facebook, "Facebook")
plot_with_matlab(eng, degree_facebook, lcc_facebook, "Facebook")

--- Facebook ---
n (Number of nodes): 4039
m (Number of edges): 88234
min(d) (Minimum degree): 1
max(d) (Maximum degree): 1045
avg(d) (Average degree): 43.69
Degree distribution type: Scale-free
Diameter: Good



In [4]:

file_path_dblp = 'data/COM-DBLP/com-dblp.ungraph.txt'
degree_dblp, lcc_dblp = analyze_graph(file_path_dblp, "DBLP")
plot_with_matlab(eng, degree_dblp, lcc_dblp, "DBLP")



--- DBLP ---
n (Number of nodes): 317080
m (Number of edges): 1049866
min(d) (Minimum degree): 1
max(d) (Maximum degree): 343
avg(d) (Average degree): 6.62
Degree distribution type: Scale-free
Diameter: Good



In [5]:
eng.exit()

### Q2

In [2]:
# Helper function to load G_1


def load_g1(file_path):
    # Load the undirected graph using networkx
    G_nx = nx.read_edgelist(file_path, create_using=nx.Graph(), nodetype=int)
    return G_nx


# New function to compute the G_2 graph from G_1
def construct_g2_sparse(G_nx):
    # Compute the sparse adjacency matrix of G_1
    A1_sparse = nx.to_scipy_sparse_array(G_nx)  # This is the correct method
    
    # Compute A_2 = A_1^2 (2-step connectivity) using sparse matrix multiplication
    A2_sparse = A1_sparse.dot(A1_sparse)

    # Create the G_2 graph from the sparse matrix
    G2_nx = nx.from_scipy_sparse_array(A2_sparse)

    return G2_nx


# Helper function to analyze a graph (G_2)
def analyze_g2(G2_nx, network_name):
    # Degree statistics
    degrees = np.array([deg for (node, deg) in G2_nx.degree()])
    max_degree = np.max(degrees)
    min_degree = np.min(degrees)
    avg_degree = np.mean(degrees)

    # # Degree distribution type (simplified: scale-free or unknown)
    # if np.max(degrees) > 10 * np.mean(degrees):  # rough heuristic for scale-free
    #     degree_dist_type = 'Scale-free'
    # else:
    #     degree_dist_type = 'Unknown'

    # Step 3: Check for connectivity (undirected graph)
    # is_connected = nx.is_connected(G2_nx)
    # diameter = nx.diameter(G2_nx) if is_connected else 'N/A - Graph not connected'

    # Print out the required information
    print(f"--- {network_name} (G2) ---")
    print(f"min(d) (Minimum degree): {min_degree}")
    print(f"max(d) (Maximum degree): {max_degree}")
    print(f"avg(d) (Average degree): {avg_degree:.2f}")
    # print(f"Degree distribution type: {degree_dist_type}")
    # print(f"Diameter: {diameter}")
    print()

    # Step 4: Compute the Local Clustering Coefficient (LCC) for each node
    lcc_dict = nx.clustering(G2_nx)
    lcc_values = np.array(list(lcc_dict.values()))  # Convert to numpy array

    return degrees, lcc_values


# Helper function to plot distributions using MATLAB
def plot_with_matlab(eng, degree, lcc_values, network_name, graph_label="G2"):
    nbins = 50

    # Plot Degree Distribution
    eng.workspace['d'] = degree
    figID = 1
    eng.eval(f"plot_distribution(d, '{network_name} Degree Distribution ({graph_label})', {nbins}, {figID})", nargout=0)

    # Plot LCC Distribution
    eng.workspace['lcc'] = lcc_values
    figID = 2
    eng.eval(f"plot_distribution(lcc, '{network_name} LCC Distribution ({graph_label})', {nbins}, {figID})", nargout=0)


eng = matlab.engine.start_matlab()
eng.eval("addpath(genpath('Mcodes'))", nargout=0)

In [7]:

file_path_amazon = 'data/com-amazon.ungraph/com-amazon.ungraph.txt'
G1_amazon = load_g1(file_path_amazon)  # Load G1 (recompute)

G2_amazon = construct_g2_sparse(G1_amazon)  # Construct G2 from G1
degree_amazon_g2, lcc_amazon_g2 = analyze_g2(G2_amazon, "Amazon")  # Analyze G2

# Plot results for G2 (Amazon)
plot_with_matlab(eng, degree_amazon_g2, lcc_amazon_g2, "Amazon", "G2")

--- Amazon (G2) ---
min(d) (Minimum degree): 3
max(d) (Maximum degree): 1328
avg(d) (Average degree): 42.15



In [6]:

file_path_facebook = 'data/facebook/facebook_combined.txt'
G1_facebook = load_g1(file_path_facebook)  # Load G1 (recompute)

G2_facebook = construct_g2_sparse(G1_facebook)  # Construct G2 from G1
degree_facebook_g2, lcc_facebook_g2 = analyze_g2(G2_facebook, "Facebook")  # Analyze G2

# Plot results for G2 (Facebook)
plot_with_matlab(eng, degree_facebook_g2, lcc_facebook_g2, "Facebook", "G2")

--- Facebook (G2) ---
min(d) (Minimum degree): 58
max(d) (Maximum degree): 2916
avg(d) (Average degree): 718.13



In [3]:

file_path_dblp = 'data/COM-DBLP/com-dblp.ungraph.txt'
G1_dblp = load_g1(file_path_dblp)  # Load G1 (recompute)

G2_dblp = construct_g2_sparse(G1_dblp)  # Construct G2 from G1
degree_dblp_g2, lcc_dblp_g2 = analyze_g2(G2_dblp, "DBLP")  # Analyze G2

# Plot results for G2 (DBLP)
plot_with_matlab(eng, degree_dblp_g2, lcc_dblp_g2, "DBLP", "G2")

--- DBLP (G2) ---
min(d) (Minimum degree): 3
max(d) (Maximum degree): 5396
avg(d) (Average degree): 88.07



In [ ]:

eng.exit()

### Q3

In [11]:
# See .mlx

### Q4

In [11]:
from src.pyC4H import *
from src.pyP2G import *
from src.whuHIloader import *
import matlab.engine

# Helper function to plot distributions using MATLAB
def plot_with_matlab(eng, degree, lcc_values=None, network_name=None, graph_label="G2"):
    nbins = 50

    # Plot Degree Distribution
    eng.workspace['d'] = degree
    figID = 1
    eng.eval(f"plot_distribution(d, '{network_name} In Degree Distribution ({graph_label})', {nbins}, {figID})", nargout=0)
    
    # Plot LCC Distribution
    eng.workspace['lcc'] = lcc_values
    figID = 2
    eng.eval(f"plot_distribution(lcc, '{network_name} aLCC Distribution ({graph_label})', {nbins}, {figID})", nargout=0)
    
def fast_local_clustering_coefficient(A):
    """
    Fast calculation of the local clustering coefficient using A^3.
    This method works for both weighted and unweighted adjacency matrices.
    """
    # Ensure A is in CSR format for efficient row operations
    if not isinstance(A, csr_matrix):
        A = csr_matrix(A)

    # Step 1: Compute A^3 (A dot A dot A)
    A_cubed = A.dot(A).dot(A)

    # Step 2: Extract diagonal of A^3 (number of triangles passing through each node)
    triangles = A_cubed.diagonal()

    # Step 3: Calculate node degrees (number of neighbors)
    degrees = np.array(A.sum(axis=1)).flatten()

    # Step 4: Calculate local clustering coefficient for each node
    local_clustering = []
    for i in range(A.shape[0]):
        k_i = degrees[i]  # Degree of node i
        if k_i < 2:
            local_clustering.append(0)
            continue
        # Possible triangles is k_i * (k_i - 1) / 2
        possible_triangles = (k_i * (k_i - 1)) / 2
        # Actual triangles come from diag(A^3)
        local_clustering.append(triangles[i] / possible_triangles if possible_triangles > 0 else 0)

    return np.array(local_clustering)

method_name = 'kNN'
feature_para_pair = [None]
param_grid = {
    'spatial_embedding': [True],
    'pca_components': [12],
    'ksize': [None],
    'n_neighbors': [100]  # k = 100 for kNN
}
k = 44

eng = matlab.engine.start_matlab()
eng.eval("addpath(genpath('Mcodes'))", nargout=0)

In [12]:

dataset_name = 'WHU_Hi_HongHu'
hsi_data, gt = whuHi_load(f"data/Matlab_data_format/Matlab_data_format/WHU-Hi-HongHu/Training samples and test samples", f"data/Matlab_data_format/Matlab_data_format/WHU-Hi-HongHu/{dataset_name}_gt", var_header='HHCY')
ground_truth = gt.reshape(-1)
process_dataset(dataset_name, hsi_data, ground_truth, method_name, param_grid, feature_para_pair, no_tuning=False, threshold=0.017)

# Load the data from CSV files
distances_file = 'WHU_Hi_HongHu_knn_spatial_embedding_pca_12_no_kernel_distances.csv'
indices_file = 'WHU_Hi_HongHu_knn_spatial_embedding_pca_12_no_kernel_indices.csv'
dataset_name = 'WHU_Hi_HongHu'
hsi_data, gt = whuHi_load(f"data/Matlab_data_format/Matlab_data_format/WHU-Hi-HongHu/Training samples and test samples", f"data/Matlab_data_format/Matlab_data_format/WHU-Hi-HongHu/{dataset_name}_gt", var_header='HHCY')
ground_truth = gt.reshape(-1)
# Read the CSV files
distances = pd.read_csv(distances_file, header=None, low_memory=False)
indices = pd.read_csv(indices_file, header=None, low_memory=False)
indices = np.array(indices)
distances = np.array(distances)

# Use the existing functions to create the k-nearest neighbors graph
A = create_knn_graph(indices, distances, k)

# Apply the assertion function to the generated matrix
try:
    assert_graph_requirements(A, k)
except AssertionError as e:
    print(e)
    print("Matrix A:\n", A)
    print("Non-zero elements per column:", (A > 0).sum(axis=0))
    
# Calculate in-degree by counting non-zero entries in each column
in_degrees = np.sum(A > 0, axis=1)
local_density = fast_local_clustering_coefficient(A)
plot_with_matlab(eng, in_degrees, local_density, network_name=f"WHU-Hi-HongHu")

Processing kNN for WHU_Hi_HongHu:   0%|          | 0/1 [00:00<?, ?it/s]

Sparsity of the adjacency matrix: 0.9999014557670772
1. All weights are positive: PASS
2. Each column has exactly 44 non-zero elements: PASS
3. Matrix is column-stochastic: PASS


In [13]:
#
#
#

In [14]:
dataset_name = 'SalinasA'
data_path = 'data/SalinasA/SalinasA_corrected.mat'
var_name = 'salinasA_corrected'
gt_path = 'data/SalinasA/SalinasA_gt.mat'
gt_var_name = 'salinasA_gt'

hsi_data = load_hsi_data(data_path, var_name)
ground_truth = load_hsi_data(gt_path, gt_var_name).reshape(-1)

process_dataset(dataset_name, hsi_data, ground_truth, method_name, param_grid, feature_para_pair, no_tuning=False, threshold=0.03)
# Load the data from CSV files
distances_file = 'SalinasA_knn_spatial_embedding_pca_12_no_kernel_distances.csv'
indices_file = 'SalinasA_knn_spatial_embedding_pca_12_no_kernel_indices.csv'

# Read the CSV files
distances = pd.read_csv(distances_file, header=None, low_memory=False)
indices = pd.read_csv(indices_file, header=None, low_memory=False)
indices = np.array(indices)
distances = np.array(distances)

# Use the existing functions to create the k-nearest neighbors graph
A = create_knn_graph(indices, distances, k)

# Apply the assertion function to the generated matrix
try:
    assert_graph_requirements(A, k)
except AssertionError as e:
    print(e)
    print("Matrix A:\n", A)
    print("Non-zero elements per column:", (A > 0).sum(axis=0))
    
# Calculate in-degree by counting non-zero entries in each column
in_degrees = np.sum(A > 0, axis=1)
local_density = fast_local_clustering_coefficient(A)

plot_with_matlab(eng, in_degrees, local_density, network_name=f"SalinasA")

Processing kNN for SalinasA:   0%|          | 0/1 [00:00<?, ?it/s]

Sparsity of the adjacency matrix: 0.9938358083496778
1. All weights are positive: PASS
2. Each column has exactly 44 non-zero elements: PASS
3. Matrix is column-stochastic: PASS


In [15]:
#
#
#

In [16]:
import os
import pydicom
import numpy as np
from skimage.filters import gaussian
from skimage.exposure import equalize_adapthist

# Define the path to the DICOM folder
folder_path = 'data/SRS00013/'  # Update this to the path of your DICOM folder

# Function to convert time in HHMMSS.fff format to seconds
def time_to_seconds(t):
    """Converts a time string in HHMMSS.fff format to seconds."""
    hours, minutes, seconds = int(t[:2]), int(t[2:4]), float(t[4:])
    return 3600 * hours + 60 * minutes + seconds

# Step 1: Reading DICOM files and extracting relevant information
dicom_files = [f for f in os.listdir(folder_path) if f.endswith('.DCM')]

# Initialize lists to store image data, acquisition times, and positions
image_data = []
acquisition_times = []
image_positions = []
acquisition_numbers = []
instance_numbers = []

# Loop through DICOM files and process each
for file in dicom_files:
    file_path = os.path.join(folder_path, file)
    ds = pydicom.dcmread(file_path)
    
    # Preprocess the image (Gaussian filter + adaptive histogram equalization)
    processed_image = equalize_adapthist(gaussian(ds.pixel_array, sigma=1))
    image_data.append(processed_image)
    
    # Extract acquisition time and position
    acquisition_times.append(time_to_seconds(ds.AcquisitionTime))
    image_positions.append(ds.ImagePositionPatient)
    
    acquisition_numbers.append(ds.AcquisitionNumber)
    instance_numbers.append(ds.InstanceNumber)

# Convert lists to NumPy arrays
image_data = np.array(image_data)  # Shape: [T, X, Y]
image_positions = np.array(image_positions)
acquisition_times = np.array(acquisition_times)

# Step 2: Sort the data by acquisition time and position (Z-slice position)
# Combine image data, positions, and times into a single list of tuples
combined = list(zip(image_data, image_positions, acquisition_times, acquisition_numbers, instance_numbers))

# First, sort by acquisition time (T dimension)
combined.sort(key=lambda x: x[2])

# Then, sort by the Z-position (Y-coordinate of the image position, for slices)
combined.sort(key=lambda x: x[1][1])

# Separate the combined list back into individual lists
image_data_sorted, image_position_sorted, acquisition_time_sorted, acquisition_numbers_sorted, instance_numbers_sorted = zip(*combined)

# Convert sorted image data back to NumPy array
image_data_sorted = np.array(image_data_sorted)  # Now sorted by time and Z-position

# Step 3: Organize the data into a 4D data cube [Z, X, Y, T]
# Let's assume you have 50 time points per Z-slice and 20 Z-slices for example:
Z_slices = 20  # Change based on your data
time_points = len(image_data_sorted) // Z_slices  # Number of time points per Z-slice

# Initialize a 4D data cube with shape [Z, X, Y, T]
X, Y = image_data_sorted[0].shape  # Spatial dimensions
data_cube = np.zeros((Z_slices, X, Y, time_points))

# Fill the data cube with the sorted image data, and transpose the time dimension
for z in range(Z_slices):
    # Transpose the sub-array so that time points are in the last dimension
    data_cube[z, :, :, :] = np.transpose(image_data_sorted[z * time_points: (z + 1) * time_points, :, :], (1, 2, 0))

# Print the shape of the data cube
print("Data cube shape:", data_cube.shape)  # Should be [Z, X, Y, T]

# Assuming data_cube has the shape [Z, X, Y, T]
Z, X, Y, T = data_cube.shape

# Step 1: Reshape the data cube to flatten the (X, Y) dimensions into a single dimension
flattened_data_cube = np.reshape(data_cube, (Z, X * Y, T))

# Step 2: Verify the new shape
print("New shape of data cube:", flattened_data_cube.shape)  # Should be [Z, (X * Y), T]


process_dataset('SRS00013', flattened_data_cube, ground_truth, method_name, param_grid, feature_para_pair, no_tuning=False, threshold=0.017)
# Load the data from CSV files
distances_file = 'SRS00013_knn_spatial_embedding_pca_12_no_kernel_distances.csv'
indices_file = 'SRS00013_knn_spatial_embedding_pca_12_no_kernel_indices.csv'

# Read the CSV files
distances = pd.read_csv(distances_file, header=None, low_memory=False)
indices = pd.read_csv(indices_file, header=None, low_memory=False)
indices = np.array(indices)
distances = np.array(distances)

# Use the existing functions to create the k-nearest neighbors graph
A = create_knn_graph(indices, distances, k)

# Apply the assertion function to the generated matrix
try:
    assert_graph_requirements(A, k)
except AssertionError as e:
    print(e)
    print("Matrix A:\n", A)
    print("Non-zero elements per column:", (A > 0).sum(axis=0))
    
# Calculate in-degree by counting non-zero entries in each column
in_degrees = np.sum(A > 0, axis=1)
local_density = fast_local_clustering_coefficient(A)

plot_with_matlab(eng, in_degrees, local_density, network_name=f"SRS00013")

Data cube shape: (20, 256, 256, 50)
New shape of data cube: (20, 65536, 50)


Processing kNN for SRS00013:   0%|          | 0/1 [00:00<?, ?it/s]

Sparsity of the adjacency matrix: 0.9999664306640625
1. All weights are positive: PASS
2. Each column has exactly 44 non-zero elements: PASS
3. Matrix is column-stochastic: PASS


In [17]:
eng.exit()

### Q5

In [ ]:
# see .mlx